# Inferring
Inference plays a very vital role in LLM's. Inference can be:
1. Assigning labels for the input text(Classification).
2. Identify different list of topics covered in the input text.
3. Identify if a particular topic is being covered in the input text or not.

In [1]:
import json
import openai
import sys
sys.path.append('../')

# Getting credentials
from Config import openaiConfig
openai.organization = openaiConfig.OPENAI_ORGANISATION
openai.api_key = openaiConfig.OPENAI_API_KEY

In [2]:
def getCompletion(prompt, model='gpt-3.5-turbo'):
    messages = [
    {'role':'user','content':prompt}
    ]
    response = openai.ChatCompletion.create(model = model,
                                            messages = messages,
                                            temperature = 0)
    return response.choices[0].message['content']

## Product Review Text

In [3]:
prodReview = """
I ordered this as a gift for my son, so that he could use it with his \
Ash Ketchum cosplay for a convention we're going to. When I opened the \
box it came in, there was no...security anything, like you get with \
most boxes. No twist ties, no plastic, no...even tape on the outside of \
the box. It was just a cardboard box with official looking printing \
on it to make it look legit enough, and Pikachu was just loose on the \
inside. Most companies at least pack something to keep things from rolling around in the package, even if it's just a plastic backing for it to stay against. This was just...tossed in there with a small thing of instructions.

So, I put fresh batteries in it, as it doesn't come with them, and realized there wasn't any sort of security measure that some of them have like a pull-tab. I mean, I suppose that is only needed if they come with batteries, and I couldn't really pin why that confused me in this case. Like it really bothered me that this didn't come with batteries in it (more on that to come). But no big deal, I was prepared, and I gave it a good shake. It took some doing, but I realized I only had the switch pushed over to demo mode (lol why demo mode when it doesn't come with batteries?) and pushed it the rest of the way over.

For the first hour, everything was great. Granted, sometimes it needed shaken harder than my 4 year old could really muster, but it'd throw off a cute, lit, thunderbolt and be good. The vocals were perfect. No big deal.

Later that night...my kids come in. "I think he broke it because it doesn't want to light up anymore." Really? Broke a stuffed toy? I gave it a once over, checked the battery pack, the velcro hadn't been undone since I last did it. So...no idea what the issue was. I switched it off then on and shook it hard. Nothing. Pushed the paw then shook it again. There it did the sounds it does when it is 'charging'. I shook it hard enough that it made my chest muscles feel like they might tear and got a PIIIIIKKAAAAAAA-----" but the 'CHHHUUUUUUUUU" never came.

My kid's four, he just wanted it back to go to sleep with, so I let him rather than return it. Fast forward to going to Wal-Mart that week. We're walking down the aisle and here's this same Pikachu, but the packaging isn't an enclosed box, it's open with twist ties to keep it in. It's also twice this price, and you could push the paw and shake it in the store and it would work because it was in demo mode and CAME WITH BATTERIES IN IT.

My husband thought maybe we got the 'small' version, but they're both 12in ones...so it really just seems like either these are defective first editions or they're counterfeit models. So buy at your own risk.

Though the toy arrived 2 days earlier than expected, so I got to play with it myself before I gave it to her.
"""

In [5]:
print(prodReview)


I ordered this as a gift for my son, so that he could use it with his Ash Ketchum cosplay for a convention we're going to. When I opened the box it came in, there was no...security anything, like you get with most boxes. No twist ties, no plastic, no...even tape on the outside of the box. It was just a cardboard box with official looking printing on it to make it look legit enough, and Pikachu was just loose on the inside. Most companies at least pack something to keep things from rolling around in the package, even if it's just a plastic backing for it to stay against. This was just...tossed in there with a small thing of instructions.

So, I put fresh batteries in it, as it doesn't come with them, and realized there wasn't any sort of security measure that some of them have like a pull-tab. I mean, I suppose that is only needed if they come with batteries, and I couldn't really pin why that confused me in this case. Like it really bothered me that this didn't come with batteries in 

## Classify Sentiment(Positive/ Negative)

In [6]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: '''{prodReview}'''
"""
response = getCompletion(prompt)
print(response)

The sentiment of the review is negative.


In [7]:
# Sentiment as either positive or negative
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as single word, either "positive" or "negative".
Review text: '''{prodReview}'''
"""
response = getCompletion(prompt)
print(response)

negative


## Identify types of Emotions

In [8]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: '''{prodReview}'''
"""
response = getCompletion(prompt)
print(response)

confused, bothered, frustrated, disappointed, skeptical


## Identify Anger

In [9]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: '''{prodReview}'''
"""
response = getCompletion(prompt)
print(response)

No.


## Extract product and company name from customer reviews

In [11]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: '''{prodReview}'''
"""
response = getCompletion(prompt)
print(response)

{
    "Item": "Pikachu stuffed toy",
    "Brand": "unknown"
}


## Doing multiple tasks at once

In [12]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{prodReview}'''
"""
response = getCompletion(prompt)
print(response)

{
    "Sentiment": "negative",
    "Anger": false,
    "Item": "Pikachu stuffed toy",
    "Brand": "unknown"
}


## Inferring topics

In [13]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

In [14]:
# Infer 5 topics
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = getCompletion(prompt)
print(response)

government survey, job satisfaction, NASA, Social Security Administration, employee concerns


In [15]:
response.split(sep=',')

['government survey',
 ' job satisfaction',
 ' NASA',
 ' Social Security Administration',
 ' employee concerns']

## Make a news alert system

In [20]:
topics = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

In [21]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Format your response as a JSON object with \
each topic as the keys. Format your response \
as boolean.

List of topics: {", ".join(topics)}

Text sample: '''{story}'''
"""
response = getCompletion(prompt)
print(response)

{
  "nasa": true,
  "local government": false,
  "engineering": false,
  "employee satisfaction": true,
  "federal government": true
}


In [26]:
response_dict = json.loads(response)

In [29]:
if response_dict['nasa']:
    print('New NASA news out!!!')

New NASA news out!!!
